In [1]:
import sqlite3
import pandas as pd


conn = sqlite3.connect('../KnowledgeBelief/knwlg_blf.db')
dat = pd.read_sql('SELECT * FROM trials A INNER JOIN subjects B ON A.prolific_id = B.prolific_id WHERE B.completion_code="XXXX"', con=conn)

In [2]:
sdat = dat.loc[dat.session_id == '0.24811221981971243'] #.reset_index(drop=True)
sdat.sort_values(['trial_type', 'belief_type', 'ascription_type'])

,id,trial_num,correct,trial_type,scenario,belief_type,ascription_type,correct_answer,target_onset,response_onset,...,screen_width,screen_height,operating_sys,operating_sys_lang,GMT_timestamp,block1_complete,block2_complete,block3_complete,completion_code,feedback
8,9,9,NaN,distractor,6,FB,Knows,f,None,None,...,None,None,macos,None,2022-06-17 19:05:50.535940,1,1,1,XXXX,Hey good job Bryan
11,12,12,NaN,distractor,9,FB,Knows,f,None,None,...,None,None,macos,None,2022-06-17 19:05:50.535940,1,1,1,XXXX,Hey good job Bryan
13,14,14,NaN,distractor,8,FB,Thinks,j,None,None,...,None,None,macos,None,2022-06-17 19:05:50.535940,1,1,1,XXXX,Hey good job Bryan
19,20,20,NaN,distractor,10,FB,Thinks,f,None,None,...,None,None,macos,None,2022-06-17 19:05:50.535940,1,1,1,XXXX,Hey good job Bryan
14,15,15,NaN,distractor,4,IG,Knows,j,None,None,...,None,None,macos,None,2022-06-17 19:05:50.535940,1,1,1,XXXX,Hey good job Bryan
15,16,16,NaN,distractor,12,IG,Knows,j,None,None,...,None,None,macos,None,2022-06-17 19:05:50.535940,1,1,1,XXXX,Hey good job Bryan
9,10,10,NaN,distractor,2,IG,Thinks,j,None,None,...,None,None,macos,None,2022-06-17 19:05:50.535940,1,1,1,XXXX,Hey good job Bryan
22,23,23,NaN,distractor,1,IG,Thinks,j,None,None,...,None,None,macos,None,2022-06-17 19:05:50.535940,1,1,1,XXXX,Hey good job Bryan
2,3,3,NaN,distractor,7,TB,Knows,f,None,None,...,None,None,macos,None,2022-06-17 19:05:50.535940,1,1,1,XXXX,Hey good job Bryan
7,8,8,NaN,distractor,3,TB,Knows,j,None,None,...,None,None,macos,None,2022-06-17 19:05:50.535940,1,1,1,XXXX,Hey good job Bryan
